In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
from statistics import mode 
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import RandomizedSearchCV

## Optimización del modelo

In [28]:
data = pd.read_csv('database.csv', sep = ",", dtype = {"fullVisitorID":str})

In [32]:
database = data.drop(["fullVisitorID","city","date"],axis = 1)

In [33]:
database.columns

Index(['visitNumber', 'bounces', 'hits', 'pageviews', 'timeOnSite',
       'transactionRevenue', 'transactions', 'source', 'channelGrouping',
       'browser', 'deviceCategory', 'country'],
      dtype='object')

## Modelo elegido: random forest 

Razón: requiero una probabilidad, por lo que usaré un algoritmo de clasificación que tenga salida del tipo probabilística. Primeramente pensé en hacer una regresión logística, pero finalmente opté por el random forest dado que hay un desbalance importante de clases y hay muchas variables categóricas que no sé si son independientes entre sí. 

Transformación de data para hacer el modelo:

01) Variables transactions y transactionRevenue -> las reemplazo por una variable ventas (1 o 0) que es el target.

02) Variables numéricas (visitNumber, bounces, hits, pageviews, timeOnSite) -> Analizo correlación para eliminar codependencia en el modelo.

03) Variables categóricas('source', 'channelGrouping','browser', 'deviceCategory', 'country')-> Analizar grupos minoritarios para truncar opciones (si es posible), elegir cuáles variables incluir y cuales no,generar variables dummy.

Realicé algunas variantes:

01) Un modelo con todas las variables excepto el país, dado que hay muchos países con pequeñas interacciones, pero al truncar los datos la mayoría de los países quedan dentro de la categoría "otros".

02) Analicé correlación entre variabes numéricas para ver si era posible (y mejor) quitar alguna.

03) Agrego el apaís a la variación 2.

In [34]:
# Elimino información de transacciones y únicamente dejo si se vendió o no
database["ventas"] = 0
database.loc[database["transactions"] > 0,"ventas"] = 1

In [36]:
# Correlación de variables numéricas
# Calculo la matriz de correlación, para eliminar aquellas variables correlacionadas antes de hacer el modelo
database[["visitNumber","bounces","hits","pageviews","timeOnSite"]].corr()>0.75

,visitNumber,bounces,hits,pageviews,timeOnSite
visitNumber,True,False,False,False,False
bounces,False,True,False,False,False
hits,False,False,True,True,False
pageviews,False,False,True,True,False
timeOnSite,False,False,False,False,True


#### Análisis de variables categóricas 

Source, browser y country tienen muchas categorías muy poco frecuentes.
En el caso de source y de browser, agrupo entradas similares (ej: opera miny y opera) y elimino aquellos valores  pocos frecuentes (el numero usado fue elegido a partir de los datos).


In [42]:
# Source
database.loc[database["source"].apply(lambda x: ("google" in x) and ("analytics" not in x)),"source"] = "google"
database.loc[database["source"].apply(lambda x: ("yahoo" in x)),"source"] = "yahoo"
database.loc[database["source"].apply(lambda x: ("facebook" in x)),"source"] = "facebook"

a = database["source"].value_counts().reset_index()
a = a[a["source"]<500]["index"].to_list()
database.loc[database["source"].apply(lambda x: x in a ),"source"] = "other"

# Browser
database.loc[database["browser"].apply(lambda x: ("Safari" in x) and ("analytics" not in x)),"browser"] = "Safari"
database.loc[database["browser"].apply(lambda x: ("Opera" in x)),"browser"] = "Opera"
database.loc[database["browser"].apply(lambda x: ("Android" in x)),"browser"] = "Android"

a = database["browser"].value_counts().reset_index()
a = a[a["browser"]<900]["index"].to_list()
database.loc[database["browser"].apply(lambda x: x in a ),"browser"] = "other"

### Modelo A -> no incluyo el país

In [90]:
# DataSet con variables a usar
data = database[['visitNumber', 'bounces', "pageviews",'hits', 'timeOnSite',
       'transactionRevenue', 'transactions', 'source', 'channelGrouping',
       'browser', 'deviceCategory', 'ventas']]

In [91]:
data.fillna({'pageviews' : 0}, inplace = True)

/home/mariano/.pyenv/versions/3.8.1/envs/ds/lib/python3.8/site-packages/pandas/core/series.py:4460: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [92]:
# Genero variables dummy
data=pd.get_dummies(data,drop_first=True,columns=["source","browser","deviceCategory","channelGrouping"])

In [93]:
X = data[['visitNumber', 'bounces', 'hits', "pageviews",'timeOnSite', 'transactionRevenue',
       'transactions', 'source_Partners',
       'source_analytics.google.com', 'source_baidu', 'source_bing',
       'source_dfa', 'source_facebook', 'source_google', 'source_other',
       'source_qiita.com', 'source_quora.com', 'source_reddit.com',
       'source_siliconvalley.about.com', 'source_t.co', 'source_yahoo',
       'source_youtube.com', 'browser_Chrome', 'browser_Edge',
       'browser_Firefox', 'browser_Internet Explorer', 'browser_Opera',
       'browser_Safari', 'browser_UC Browser', 'browser_YaBrowser',
       'browser_other', 'deviceCategory_mobile', 'deviceCategory_tablet',
       'channelGrouping_Affiliates', 'channelGrouping_Direct',
       'channelGrouping_Display', 'channelGrouping_Organic Search',
       'channelGrouping_Paid Search', 'channelGrouping_Referral',
       'channelGrouping_Social']]

y = data["ventas"]

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [95]:
# Genero el primer modelo, sin modificar hiperparámetros 
rdf1 = RandomForestClassifier()

In [96]:
# Entreno 
rdf1.fit(X_train, y_train)

# Predicciones
predicciones = rdf1.predict_proba(X_test)[:,1]

In [1]:
# Como métrica utilizo la Puntuación de Brier
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html#sklearn.metrics.brier_score_loss
brier_score_loss(y_test,predicciones)

NameError: name 'brier_score_loss' is not defined

### Modelo C -> incluyo el país

In [65]:
# Elimino países con menos visitas que el 1% de los datos totales
a = database["country"].value_counts().reset_index()
a = a[a["country"]<350]["index"].to_list()
database.loc[database["country"].apply(lambda x: x in a ),"country"] = "other"

In [69]:
# DataSet con variables a usar
data2 = database[['visitNumber', 'bounces', 'hits', 'timeOnSite',
       'transactionRevenue', 'transactions', 'source', 'channelGrouping',
       'browser', 'deviceCategory','country', 'ventas']]

In [70]:
# Genero variables dummy
data2=pd.get_dummies(data2,drop_first=True,columns=["source","browser","deviceCategory","channelGrouping","country"])

In [76]:
y2 = data["ventas"]
X2 = data.drop("ventas", axis = 1)

In [77]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.5, random_state=42)

In [78]:
# Genero el primer modelo, sin modificar hiperparámetros 
rdf2 = RandomForestClassifier()

In [79]:
# Entreno 
rdf2.fit(X2_train, y2_train)

# Predicciones
predicciones2 = rdf2.predict_proba(X2_test)[:,1]

In [80]:
# Como métrica utilizo el Coeficiente Brier
brier_score_loss(y2_test,predicciones2)

2.570308282763573e-06

### Genero una optimización de hiperparámetros usando RandomSeach en rdf2

In [99]:
rf_random = RandomizedSearchCV(estimator = rdf2, param_distributions = random_grid, n_iter = 50, scoring = 'neg_brier_score',cv = 3, verbose=10, random_state=42, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_brier_score', verbose=10)

In [100]:
rf_random.best_score_

-2.6303825968545243e-06

In [101]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=100, max_features='sqrt',
                       min_samples_split=5, n_estimators=800)

In [102]:
rdf2b = RandomForestClassifier(bootstrap=False, max_depth=100, max_features='sqrt',
                       min_samples_split=5, n_estimators=800)

In [103]:
rdf2b.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=100, max_features='sqrt',
                       min_samples_split=5, n_estimators=800)

In [104]:
probas2 = rdf2b.predict_proba(X_test)[:,1]

In [105]:
brier_score_loss(y_test,probas2)

1.7698085165601543e-06

In [106]:
# Hago random más largo, ya la mejora anterior fue buena, pero aprovecho y le doy rosca
rf_random2 = RandomizedSearchCV(estimator = rdf2, param_distributions = random_grid, n_iter = 100, scoring = 'neg_brier_score',cv = 4, verbose=10, random_state=42, n_jobs = -1)

rf_random2.fit(X_train, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='neg_brier_score', verbose=10)

In [109]:
rf_random2.best_score_

-2.284982421325635e-06

In [110]:
rf_random2.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=80, max_features='sqrt',
                       min_samples_split=5, n_estimators=1400)

In [111]:
rdfbest = rf_random2.best_estimator_

In [112]:
rdfbest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=80, max_features='sqrt',
                       min_samples_split=5, n_estimators=1400)

In [113]:
probas4 = rdfbest.predict_proba(X_test)[:,1]

In [115]:
brier_score_loss(y_test,probas4)

1.8755656021180322e-06